In [ ]:
import requests
from serpapi import GoogleSearch
from newspaper import Article
from transformers import pipeline
import re

SERP_API_KEY = "4c3a39f406bb56f40ea28e7b2b59186206d9eb054c352ef8e19552d9900611f2"

def search_study_materials(query):
    params = {
        "q": query + " study materials",
        "hl": "en",
        "gl": "us",
        "api_key": SERP_API_KEY
    }
    search = GoogleSearch(params)
    results = search.get_dict()
    
    links = []
    for result in results.get("organic_results", []):
        if "link" in result and re.match(r'https?://', result["link"]):
            links.append(result["link"])
    
    return links[:5]

def summarize_article(url):
    try:
        article = Article(url)
        article.download()
        article.parse()
        article.nlp()
        return article.summary
    except Exception as e:
        return f"Error processing article: {str(e)}"

def rank_materials(study_links, course_objectives):
    summarizer = pipeline("summarization")
    ranked_materials = []
    
    for link in study_links:
        summary = summarize_article(link)
        relevance_score = sum(1 for obj in course_objectives if obj.lower() in summary.lower())
        ranked_materials.append((link, relevance_score, summary))
    
    ranked_materials.sort(key=lambda x: x[1], reverse=True)
    return ranked_materials

query = "Deep Learning"
course_objectives = ["Neural Networks", "Backpropagation", "Optimization"]

study_links = search_study_materials(query)
recommended_materials = rank_materials(study_links, course_objectives)

for link, score, summary in recommended_materials:
    print(f"Link: {link}\nRelevance Score: {score}\nSummary: {summary}\n\n")
